In [ ]:
#import libraries
import cv2
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.utils.np_utils import to_categorical

import io

In [ ]:
#import files
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [ ]:
uploaded

In [ ]:
# a sighn the uploded file
imgtest = np.load('testimage.npy', mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
imgtran = np.load('trainimage.npy', mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
laptest = pd.read_csv(io.StringIO(uploaded['testLabels.csv'].decode('utf-8')))
laptran = pd.read_csv(io.StringIO(uploaded['trainLabels.csv'].decode('utf-8')))

In [ ]:
print(imgtest.shape)
print(imgtran.shape)
print(laptest.shape)
print(laptran.shape)

In [ ]:
#show the trean img
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(imgtest[i])

In [ ]:
#show the test img
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(imgtran[i])

Exploring the data

In [ ]:
#take a look of test data
laptest.count

In [ ]:
 #look of trean data
laptran.count

In [ ]:
#show the number of nunique values. 
laptest.nunique()


In [ ]:
#print nunique names 
laptest.Label.value_counts()

In [ ]:
# merge the imgs & labls

#labls = pd.merge(laptest,laptran)

pre-processing

In [ ]:
# preprocessed test images
preprocessed_imgtest = []       
for img in imgtest:
    
    blurImg = cv2.GaussianBlur(img, (5, 5), 0)
    preprocessed_imgtest.append(blurImg) 
    
# Show examples
plt.subplot(1, 3, 1); plt.imshow(img)
plt.subplot(1, 3, 2); plt.imshow(blurImg)

preprocessed_imgtest = np.asarray(preprocessed_imgtest) 

In [ ]:
# preprocessed trean images
preprocessed_imgtran = []       
for img in imgtran:
    
    blurImg = cv2.GaussianBlur(img, (5, 5), 0)
    preprocessed_imgtran.append(blurImg) 
    
# Show examples
plt.subplot(1, 3, 1); plt.imshow(img)
plt.subplot(1, 3, 2); plt.imshow(blurImg)

preprocessed_imgtran = np.asarray(preprocessed_imgtran) 

In [ ]:
# Show sample result for test
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(preprocessed_imgtest[i])

In [ ]:
# Show sample result for trean
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(preprocessed_imgtran[i])

In [ ]:
# Normalize images 
preprocessed_imgtest = preprocessed_imgtest / 255
preprocessed_imgtran = preprocessed_imgtran / 255

data compatible

In [ ]:
# Convert labels from digits to one hot vectors.

from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
y_test = enc.fit_transform(laptest)
y_train = enc.fit_transform(laptran)

In [ ]:
print (y_test[0])
print (y_train[0])

In [ ]:
# Reshape data into shapes compatible with Keras models.

X_train = preprocessed_imgtran.reshape(preprocessed_imgtran.shape[0], 128, 128, 3)
X_test = preprocessed_imgtest.reshape(preprocessed_imgtest.shape[0], 128, 128, 3)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
random_seed = 2
from sklearn.model_selection import train_test_split
X_test, X_val, y_test, Y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state=random_seed)

In [ ]:
print(X_test.shape)
print(X_val.shape)

# Building CNN:

In [ ]:
# Set the CNN model

batch_size = None

model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), padding='same', activation="relu", input_shape=(128, 128, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (5, 5), padding='same', activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(12, activation="softmax"))

model.summary()

In [ ]:
# Set the optimizer and loss function, and compile the model with them.

optimizer = Adam(
                  learning_rate=0.001,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-07,
                  amsgrad=False,
                  name='Adam')
model.compile(optimizer = optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Set epochs = 10, and fit the model. 
#history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_val,Y_val),batch_size = batch_size)
optimizer

In [ ]:
# Evaluate the model.

score = model.evaluate(X_test, y_test, verbose=0, batch_size = 3)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
result = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val, axis=1)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(Y_true, result)

df_cm = pd.DataFrame(conf_mat, index = [i for i in range(0, 4)],columns = [i for i in range(0, 4)])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt='g');

In [ ]:
import numpy as np

plt.figure(figsize=(2,2))
plt.imshow(X_test[3],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[3].reshape(1,128,128,3))))
print('True Label', np.argmax(y_test[3]))

plt.figure(figsize=(2,2))
plt.imshow(X_test[2],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[2].reshape(1,128,128,3))))
print('True Label', np.argmax(y_test[2]))

plt.figure(figsize=(2,2))
plt.imshow(X_test[33],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[33].reshape(1,128,128,3))))
print('True Label', np.argmax(y_test[33]))

plt.figure(figsize=(2,2))
plt.imshow(X_test[59],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[59].reshape(1,128,128,3))))
print('True Label', np.argmax(y_test[59]))

plt.figure(figsize=(2,2))
plt.imshow(X_test[36],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[36].reshape(1,128,128,3))))
print('True Label', np.argmax(y_test[36]))